In [1]:
import json
import pandas as pd
import os

# --- 1. Load dữ liệu categories.csv ---
categories = pd.read_csv("public/polyvore_outfits/categories.csv")

In [2]:
# Chuyển NaN -> chuỗi rỗng và ép kiểu string
categories["sub_category"] = categories["sub_category"].fillna("").astype(str)
categories["main_category"] = categories["main_category"].fillna("").astype(str)

# Gộp các sub_category và main_category trùng category_id
cat_grouped = (
    categories.groupby("category_id", dropna=False)
    .agg({
        "sub_category": lambda x: ", ".join(sorted(set(map(str, x)))),
        "main_category": lambda x: ", ".join(sorted(set(map(str, x))))
    })
    .reset_index()
)

# Tạo dictionary tra cứu nhanh
cat_dict = cat_grouped.set_index("category_id")[["sub_category", "main_category"]].to_dict(orient="index")

print(f"✅ Đã đọc {len(cat_dict)} loại category hợp lệ.\n")

✅ Đã đọc 158 loại category hợp lệ.



In [3]:
# 2️⃣ ĐỌC FILE polyvore_item_metadata.json
# ============================
item_metadata_path = "public/polyvore_outfits/polyvore_item_metadata.json"

with open(item_metadata_path, "r", encoding="utf-8") as f:
    item_data = json.load(f)

print(f"✅ Đã đọc {len(item_data)} item metadata.\n")

✅ Đã đọc 251008 item metadata.



In [6]:
# 3️⃣ GHÉP DỮ LIỆU
# ============================
merged_data = []

for item_id, info in item_data.items():
    category_id_str = info.get("category_id", "-1")
    try:
        category_id = int(category_id_str)
    except ValueError:
        category_id = -1

    cat_info = cat_dict.get(category_id, {"sub_category": "", "main_category": ""})

    # Tạo đường dẫn ảnh
    image_path_jpg = os.path.join("pulic/polyvore_outfits", "images", f"{item_id}.jpg")
    image_path_png = os.path.join("public/polyvore_outfits", "images", f"{item_id}.png")

    if os.path.exists(image_path_jpg):
        image_path = image_path_jpg
    elif os.path.exists(image_path_png):
        image_path = image_path_png
    else:
        image_path = None

    merged_data.append({
        "item_id": item_id,
        "title": info.get("title", ""),
        "description": info.get("description", ""),
        "url_name": info.get("url_name", ""),
        "category_id": category_id,
        "sub_category": cat_info["sub_category"],
        "main_category": cat_info["main_category"],
        "image_path": image_path
    })

print(f"✅ Đã gộp xong {len(merged_data)} items.\n")

✅ Đã gộp xong 251008 items.



In [7]:
# 4️⃣ LƯU RA FILE JSON
# ============================
output_path = "public/polyvore_outfits/polyvore_merged_items.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)

print(f"🎉 File JSON cuối cùng đã được tạo: {output_path}")
print("📄 Ví dụ 1 item đầu tiên:")
print(json.dumps(merged_data[0], indent=4, ensure_ascii=False))

🎉 File JSON cuối cùng đã được tạo: public/polyvore_outfits/polyvore_merged_items.json
📄 Ví dụ 1 item đầu tiên:
{
    "item_id": "211990161",
    "title": "",
    "description": "",
    "url_name": "neck print chiffon plus size",
    "category_id": 15,
    "sub_category": "tunic",
    "main_category": "tops",
    "image_path": null
}
